# Fetch Rewards Coding Exercise - Analytics Engineer

## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

The aim is to convert the unstructured JSON files into cleaned structured data which we can query. I will use pandas to process and clean the data.

In [1]:
import gzip
import shutil
import os
import pandas as pd
from ast import literal_eval
import json
from datetime import datetime
from sqlalchemy import create_engine


The first step is to read the zipped json files through python.

In [2]:
for i in os.listdir():
  if 'json' in i:
    with gzip.open(i, 'rb') as f_in:
          with open(i.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
receipts = pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)

### Receipts table

In [4]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


The next step is to explode the 'rewardsReceiptItemList' field as it contains lists of receipts(we do this using literal_eval function from the ast package). Then we need to split each dictionary and get the columns from it(we do this using the json_normalize function in pandas). Once we do this we join this data back to the receipts data frame.

In [5]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [6]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [7]:
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

In [8]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')

In [9]:
receipts_final

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


Now that we have exploded 'rewardsReceiptItemList' into multiple rows and split the keys into different columns we need to format the other fields that have dictionaries.

In [10]:
set().union(*(d.keys() for d in receipts_final['_id']))

{'$oid'}

In [11]:
set().union(*(d.keys() for d in receipts_final['createDate']))

{'$date'}

In [12]:
set().union(*(d.keys() for d in receipts_final['dateScanned']))

{'$date'}

In [13]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['finishedDate'])['finishedDate']))

{'$date'}

In [14]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['modifyDate'])['modifyDate']))

{'$date'}

In [15]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['pointsAwardedDate'])['pointsAwardedDate']))

{'$date'}

In [16]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['purchaseDate'])['purchaseDate']))

{'$date'}

We see that each of these columns contain only one dictionary item each so we just need to extract these fields. As for the date columns the time is stored as UTC so we convert that into datetime format using the apply function.

In [17]:
def date_converter(x):
  try:
    return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except TypeError:
    return(None)


In [18]:
receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])
receipts_final['createDate'] = receipts_final['createDate'].apply(lambda x: date_converter(x))
receipts_final['dateScanned'] = receipts_final['dateScanned'].apply(lambda x: date_converter(x))
receipts_final['finishedDate'] = receipts_final['finishedDate'].apply(lambda x: date_converter(x))
receipts_final['modifyDate'] = receipts_final['modifyDate'].apply(lambda x: date_converter(x))
receipts_final['pointsAwardedDate'] = receipts_final['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts_final['purchaseDate'] = receipts_final['purchaseDate'].apply(lambda x: date_converter(x))

In [19]:
receipts_final.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Users table

In [20]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [21]:
set().union(*(d.keys() for d in users['_id']))

{'$oid'}

In [22]:
set().union(*(d.keys() for d in users['createdDate']))

{'$date'}

In [23]:
set().union(*(d.keys() for d in users.dropna(subset=['lastLogin'])['lastLogin']))

{'$date'}

Here we see that each dictionary contains just one key each and we just need to extract the value and we will use the same function created earlier to convert date into a user friendly format.

In [24]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [25]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


## Brands table

In [26]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [27]:
set().union(*(d.keys() for d in brands['cpg']))

{'$id', '$ref'}

We need to explode the cpg fields as there is more than one element in the dictionary.

In [28]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')

In [29]:
brands_norm

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [30]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

In [31]:
brands_final.shape

(1167, 10)

In [32]:
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])

In [33]:
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


## Second: Write a query that directly answers a predetermined question from a business stakeholder

I am using SQLite database through [SQL Alchemy](https://docs.sqlalchemy.org/en/14/dialects/sqlite.html) python to implement the SQL database for the queries.

In [34]:
engine = create_engine('sqlite://', echo=False)

In [35]:
users.drop_duplicates(subset=['_id']).to_sql('users',con=engine)

In [36]:
receipts_final.columns = receipts_final.columns.str.replace(".", "_")

C:\Users\pares\AppData\Local\Temp/ipykernel_28360/2797878528.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  receipts_final.columns = receipts_final.columns.str.replace(".", "_")


In [37]:
receipts_final.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=engine)

In [38]:
brands_final.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands',con=engine)

Which brand has the most spend among users who were created within the past 6 months?

In [39]:
engine.execute("SELECT * from (SELECT r.rewardsReceiptItemList_brandCode,SUM(r.totalSpent) as summedTotalSpent from receipts r where r.userId in (SELECT _id from users u where u.createdDate > (SELECT strftime('%Y-%m-%d %H:%M:%S', 'now','-6 month'))) group by r.rewardsReceiptItemList_brandCode) where rewardsReceiptItemList_brandCode is not NULL order by summedTotalSpent DESC LIMIT 1;").fetchall()

[]

## Third: Evaluate Data Quality Issues in the Data Provided

To find data quality issues I decided to look into the mapping between the brands data and receipts data using brandCode.

In [47]:
len(set(receipts_final.dropna(subset=['rewardsReceiptItemList_brandCode'])['rewardsReceiptItemList_brandCode']))

227

In [41]:
len(set(brands_final.dropna(subset=['brandCode'])['brandCode']))

897

As we see there are a lot more unique brand codes in the brands dataset compared to the receipts. Now we compare the two and see if all the receipts with a brand code can be mapped to the brands table.

In [42]:
receipts_dropped = receipts_final.dropna(subset=['rewardsReceiptItemList_brandCode'])
receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))

C:\Users\pares\AppData\Local\Temp/ipykernel_28360/2750043545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))


In [43]:
brands_final['barcode'] = brands_final['barcode'].apply(lambda x: str(x))
brands_final['brandCode'] = brands_final['brandCode'].apply(lambda x: str(x))

In [44]:
receipts_with_brands = pd.merge(receipts_final,brands_final,left_on='rewardsReceiptItemList_brandCode',right_on='brandCode')

In [45]:
set(receipts_dropped[~(receipts_dropped['rewardsReceiptItemList_brandCode'].isin(brands['brandCode']))]['rewardsReceiptItemList_brandCode'])

{'7UP',
 'ADVIL',
 'AMERICAN BEAUTY',
 'ARROWHEAD',
 'AZTECA',
 'BANZA',
 'BEAR CREEK COUNTRY KITCHENS',
 'BEN AND JERRYS',
 'BETTY CROCKER',
 'BIC',
 'BIGELOW',
 'BLUE DIAMOND',
 "BOAR'S HEAD",
 'BORDEN',
 'BOTA BOX',
 'BRAND',
 "BRASWELL'S",
 'BUNNY',
 "BUSH'S BEST",
 'C&H',
 'CADBURY',
 'CAL-ORGANIC FARMS',
 'CALIFIA FARMS',
 "CAMPBELL'S",
 'CARAMELLO',
 'CHEERIOS',
 'CHEESE',
 'CHEEZ-IT',
 'CHEX',
 'CHICKEN OF THE SEA',
 'CHIQUITA',
 'CINNAMON TOAST CRUNCH',
 'COKE',
 'COLEMAN NATURAL',
 "CONNIE'S PIZZA",
 'CREST 3D WHITE',
 'CRISPIX',
 'DANNON',
 'DARE',
 'DELI',
 'DIET COKE',
 'DIGIORNO',
 'DOLE',
 'DR PEPPER',
 'EDWARDS',
 "EGGLAND'S BEST",
 'EGGO',
 'EL MONTEREY',
 'ENERGIZER MAX',
 'ESSENTIAL EVERYDAY',
 'FAGE',
 "FAMOUS DAVE'S",
 "FLORIDA'S NATURAL",
 'FOLGERS',
 'FORTUNE YAKISOBA',
 'FRANZ',
 "FRENCH'S",
 'FRESH EXPRESS',
 'FRESH STEP',
 'FRONTERA',
 'GALLO FAMILY VINEYARDS',
 'GENERAL MILLS',
 'GERBER',
 'GERM-X',
 'GREEN GIANT',
 'GRIMMWAY FARMS',
 'HANOVER',
 'HARVEST SNA

Above are all the receipts with a brand code but the codes are missing in the brands table. This would be an issue as a lot of data will be missing when we query between the two tables.